**NORMALIZACIÓN DE DATOS**

La normalización o escalamiento de datos es un proceso fundamental que se hace en el procesamiento de datos y antes de la aplicación del modelo.



 Consiste en ajustar los valores de las características en un rango específico para que todas las características tengan un impacto similar en el modelo de machine learning. Considerando la explicación vista en clase, se pueden definir que los la importancia del proceso de normalización de datos radica en los siguientes aspectos:

1.    Evitar sesgos por la escala de las características: Si las características tienen escalas muy diferentes, aquellas con valores más altos pueden dominar el modelo sobre las características con valores más bajos. Esto puede resultar en un sesgo en la importancia relativa de las características y afectar negativamente el rendimiento del modelo.

2.   Mejorar la convergencia de los algoritmos de optimización: Muchos algoritmos de machine learning se basan en técnicas de optimización que buscan minimizar una función de costo. Si las características no están normalizadas, algunas características con valores más grandes pueden tener un impacto desproporcionado en la actualización de los parámetros del modelo durante el entrenamiento. Esto puede hacer que el algoritmo de optimización converja más lentamente o se atasque en mínimos locales.

3. Facilitar la interpretación de los coeficientes: En modelos lineales, como la regresión lineal, los coeficientes se utilizan para interpretar la importancia relativa de cada característica en la predicción del resultado. Si las características no están normalizadas, los coeficientes pueden ser difíciles de interpretar, ya que su magnitud dependerá de la escala original de las características.



En este Notebbok vamos a analizar la importancia del escalamiento, considerando el algortimo de regresión lineal. Recuerde que el modelo de regresión lineal esta definido de la siguiente forma:

$\hat{y}=w_{1}x_{1}+w_{2}x_{2}+w_{3}x_{3}.....+w_{n}x_{n}+w_{0}$


Haremos uso de una base de datos que ya hemos trabajado, la cual relacióna el peso y la altura con el IMC

In [ ]:
from google.colab import drive
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import sys
drive.mount('/content/drive')
df=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Auto 1/bmi.csv')
#df=df.drop(labels='Unnamed: 0',axis=1)
df

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,Gender,Height,Weight,Index
0,Male,174,96,4
1,Male,189,87,2
2,Female,185,110,4
3,Female,195,104,3
4,Male,149,61,3
...,...,...,...,...
495,Female,150,153,5
496,Female,184,121,4
497,Female,141,136,5
498,Male,150,95,5


Se hace la transformación de datos según la necesidad

In [ ]:
df['Height']=df['Height']/100
df['Index']=df['Weight']/(df['Height']**2)
df=df.drop(labels='Gender',axis=1)


In [ ]:
df

,Height,Weight,Index
0,1.74,96,31.708284
1,1.89,87,24.355421
2,1.85,110,32.140248
3,1.95,104,27.350427
4,1.49,61,27.476240
...,...,...,...
495,1.50,153,68.000000
496,1.84,121,35.739603
497,1.41,136,68.407022
498,1.50,95,42.222222


Lo anterior será considerado como su base de datos real, a la cual hay que aplicarle un poceso de escalamiento o normalización debido a que la escala entre "Height"y "Weight" es notablemente diferente.

Existen varias formas de hacer el escalamiento de caracteristicas. Generalmente se busca que las caracteristicas se establezcan en un rango entre [0 ,1], pero puede variar a un diferentes escalas [1,-1], [3, -3], entre otros.

Una de las técnicas mas comunes de escalamiento es denominada "MinMaxScaler", el cual utiliza los maximos y minimos de las caracteristicas para hacer el escalamiento. La Función de "MinMaxScaler" se representa de la siguiente forma:

$X_{Norm}=\frac{X-X_{min}}{X_{max}-X_{min}}$

A continuación se aplicará esta normalización para definir una base de datos nombrada como "x_norm"


In [ ]:
x = df.iloc[:,:-1]
y=  df.iloc[:,-1] #NO SE NORMALIZA
x_norm= (x-x.min())/(x.max()-x.min())
#x.min()

In [ ]:
x_norm

,Height,Weight
0,0.576271,0.418182
1,0.830508,0.336364
2,0.762712,0.545455
3,0.932203,0.490909
4,0.152542,0.100000
...,...,...
495,0.169492,0.936364
496,0.745763,0.645455
497,0.016949,0.781818
498,0.169492,0.409091


Tengan en cuenta que si normalizamos los datos en todo el conjunto de datos, incluyendo tanto el conjunto de entrenamiento como el conjunto de prueba, estaríamos utilizando información del conjunto de prueba para calcular los parámetros de normalización. Esto podría resultar en una "fuga de información" donde el modelo se beneficia de información que no estaría disponible en un escenario de producción real. La normalización debe basarse únicamente en la información del conjunto de entrenamiento para evitar esta fuga de información.



En resumen, la normalización o escalamiento de datos es importante para garantizar un rendimiento óptimo del modelo, evitar sesgos por la escala de las características, mejorar la convergencia de los algoritmos de optimización, facilitar la interpretación de los coeficientes y preparar los datos para algoritmos sensibles a la escala.






In [ ]:
# Training function: fit
def training(X, y, epochs):
    m, n = X.shape #Se extraen las dimensiones de los datos
    lr=0.1        #Se define la tasa de aprendizaje
    weights = np.zeros((n,1)) #Se inicializan los pesos y el bias
    bias = 0                 #Pueden ser inicializados en 0 o de forma aleatoria

    y = y.reshape(m,1) #Se hace un reshape debido a que la dimensión original
                       #es de (m,) y esto puede ocasionar inconvenientes en las operaciones
    losses = []         #Se crea una lista vacia para hacer la estimación del costo


    for epoch in range(epochs): #Bucle para el ajuste de parametros
        y_hat = np.dot(X, weights) + bias #Se hace la predicción con el w y bias inicializado
        loss = np.mean((y_hat - y)**2) #se calcula el error mediante Mean Square Error
        losses.append(loss) #Se añade el error calculado a la lista losses
        dw1 = (1/m)*np.dot(X[:,0].T, (y_hat - y)) #se calcula la derivada con respecto a w
        dw2 = (1/m)*np.dot(X[:,1].T, (y_hat - y)) #se calcula la derivada con respecto a w
        db = (1/m)*np.sum((y_hat - y)) #Se calcula la derivada con respecto al bias
        # Updating the parameters: parameter := parameter - lr*derivative
        # of loss/cost w.r.t parameter)
        weights[0] -= lr*dw1 #se hace la estimación del nuevo parametro haciendo
        weights[1] -= lr*dw2
        bias -= lr*db     #parametro= parametro - lr*derivative
    #el bucle itera el número de veces que el usuario especifique en epochs
    return weights, bias, losses #la función retorna los ultimos pesos actualizados
                                 #el bias y la lista losses que tiene el costo en cada iteración



In [ ]:
def predict(X, weights, bias):  #La función recibe los datos, los pesos w y el bias
      #print(X.shape)
      #print(weights.shape)
      #print(np.dot(X, weights))
      return np.dot(X, weights)+ bias  #aplica un producto punto entre los datos y el bias
                                        #y retorna una estimación

In [ ]:
x = np.array(df.iloc[:,:-1])
y=  np.array(df.iloc[:,-1])
x_train, x_test, y_train, y_test = x[:400,:], x[400:,:], y[:400], y[400:] #Se hace una división de datos
print(x_train.shape, y_train.shape)
print(np.amin(x_train, axis=0))
x_train_norm = (x_train-np.amin(x_train, axis=0))/(np.amax(x_train, axis=0)-np.amin(x_train, axis=0))
x_test_norm = (x_test-np.amin(x_train, axis=0))/(np.amax(x_train, axis=0)-np.amin(x_train, axis=0))
w, b, l  = training(x_train_norm, y_train, epochs=1000) #se entrena con los datos de entrenamiento

(400, 2) (400,)
[ 1.4 50. ]


In [ ]:
l

[1575.4062244423885,
 1174.415925977945,
 887.113546673019,
 681.0758284633505,
 533.1294235505997,
 426.71197635381844,
 349.9860662820912,
 294.49122654348326,
 254.18053047675153,
 224.73203156483018,
 203.0566437142744,
 186.9464190650772,
 174.82316911080926,
 165.5588020371437,
 158.3469162525353,
 152.6110271561858,
 147.9389759930006,
 144.0360512694754,
 140.69148419666234,
 137.75450266463423,
 135.1172167812767,
 132.70238698875744,
 130.45468180031932,
 128.33442959891528,
 126.31315296355758,
 124.37037698387842,
 122.49134810538271,
 120.66540373920535,
 118.8848069792585,
 117.14391373604431,
 115.43857745193407,
 113.76572361838389,
 112.1230456524734,
 110.50878751041161,
 108.92158829311202,
 107.36037115844024,
 105.82426390022496,
 104.31254216017467,
 102.82458881611909,
 101.35986493199606,
 99.91788897149804,
 98.49822191820303,
 97.1004566174901,
 95.7242101361651,
 94.36911827922808,
 93.0348316487214,
 91.72101280506381,
 90.42733421668254,
 89.15347677338596,

In [ ]:
w

array([[-26.20197839],
       [ 39.18994795]])

In [ ]:
y_test_preds = predict(x_test_norm, w, b) #Predecir datos de entrenamiento
compare = pd.DataFrame({'Actual':y_test, 'Predicted':list(y_test_preds)})
print(compare)

       Actual             Predicted
0   27.350427  [25.830029906937117]
1   35.341525   [36.22497457192487]
2   21.132713  [19.490090541341807]
3   38.514058   [39.51429797767655]
4   34.900266   [35.98131117042515]
..        ...                   ...
95  68.000000   [63.27193014846521]
96  35.739603   [36.77177284202422]
97  68.407022  [61.212213787423195]
98  42.222222   [42.60813941181735]
99  43.770256  [45.219609998550375]

[100 rows x 2 columns]


**TAREA 2:**

1.

2.

3. Aplicar la normalización de datos en términos de datos de entrenamiento

4. Haga la implementación del algoritmo de regresión lineal, y haga la posterior evaluación del modelo con los datos de test**

5. consultar y extraerlas minimo dos metricas de desempeño en regresión y explicar su significado